In [39]:
import config
import requests
import praw
import pandas as pd
import pprint

In [40]:
user_agent = "python:ministryofedCSC_redditscraper:v0.0.1 (by rahm)"

In [41]:
reddit = praw.Reddit(client_id=config.client_id,
                     client_secret=config.client_secret,
                     user_agent=user_agent,
                    )

In [42]:
def extractSubmissionData(rs):
    """
    rs: being a reddit submission type
    return: list of attributes from submission
    """
    
    author_name = "[deleted]" if rs.author is None else rs.author.name
    
    return [rs.name, rs.title, rs.subreddit.display_name, author_name,\
            rs.score, rs.upvote_ratio, rs.num_comments, rs.selftext,\
            rs.url, rs.created_utc]

In [43]:
submissionsCol = ["id", "title", "subreddit", "author",\
                 "score", "upvote_ratio", "num_comments",\
                 "selftext", "url", "created_utc"]
totalSubmissionsData = pd.DataFrame([], columns=submissionsCol)


### Get submissions from Reddit

In [81]:
# query parameters
subToSearch = 'ontario'
sort = 'new'
time_filter = 'all'
# limit how many reddit posts you want
limit = 50
# what to search on
#query = 'title:education'
query = ''


#pd.DataFrame([], columns=submissionsCol)

def getSubmissions(subToSearch, query='', sort='new', time_filter='all', numSubmissions=30):
    # limit is a constant defined as the total number of submissions that a single reddit api call can retrieve
    # its value is 1000 according to the documentation.
    # this shouldnt be changed unless the reddit api itself is updated
    limit = 1000
    
    
    lastSubmission = None
    submissionsData = []
    i = 0
    while numSubmissions > 0:
        if numSubmissions > limit:
            # max limit is 100, so if larger, set a throttled limit
            # and then save the difference
            thrLimit = limit
            numSubmissions -= limit
        else:
            thrLimit = numSubmissions
            numSubmissions -= limit

        # additional query params that get passed to an internal class
        # 'praw.models.ListingGenerator'
        queryParams = {}
        if lastSubmission is not None:
            queryParams.update({"after": lastSubmission}) #if lastSubmission != None else pass

        if query is None:# or query == '':
            submissions = reddit.subreddit(subToSearch).new(limit=thrLimit, params=queryParams)

        else:
            submissions = reddit.subreddit(subToSearch)\
                        .search(query, sort=sort, time_filter=time_filter,\
                               limit=thrLimit, params=queryParams)
        for sub in submissions:
            print(sub.id)
            print(sub.title)
            attrs = extractSubmissionData(sub)
            submissionsData.append(attrs)
            # we can use this last submission id as a starting point for our next batch of submissions
            lastSubmission = sub.name
            i+= 1
            print(i)
    return submissionsData
        

In [83]:
getSubmissions('ontario', query='title:', sort='new', time_filter='all', numSubmissions=30)

eaejsj
Titles of high school history textbooks?
1
e7gf9a
Does anyone know if I can skip the wait period on a G2 test with a UK provisional licence
2
dz4s2z
Looking for a decent apartment to move into
3
dwiszq
SO does snow removal for a private company in Ottawa and they consistently make them drive 20-25 hours straight with no rest. Is this legal?
4
dwamll
Cell phone suction mount reco's
5
dvkpiy
Toronto family fears for safety of missing daughter. My friend is missing, only put title because the rules say it's necessary, please help
6
duaoyp
Is it just me or does zehrs really suck now?
7
dt3ka8
Removing my name off a car I don't own anymore but MTO still has me as its owner.
8
dobyrk
/r/Ontario Halloween Movie Recommendation
9
diub83
Interested in Getting a family doctor, what is the quickest way i could get one?
10
dih8ri
How to fight a speeding ticket in Owen Sound, Ontario, Canada?
11
dh0s8w
Landlord needs some advice.
12
df5m4v
Can I go to university after college?
13
de6w28
Ontar

[['t3_eaejsj',
  'Titles of high school history textbooks?',
  'ontario',
  'Fenderben',
  0,
  0.36,
  2,
  "I've been out of high school for awhile and am interested in knowing the titles of the history textbooks they're using nowadays.",
  'https://www.reddit.com/r/ontario/comments/eaejsj/titles_of_high_school_history_textbooks/',
  1576293889.0],
 ['t3_e7gf9a',
  'Does anyone know if I can skip the wait period on a G2 test with a UK provisional licence',
  'ontario',
  'Carboncat',
  0,
  0.4,
  14,
  "Just had my test for my full licence cancelled before my flights out to Canada, I'm moving there in 8 days for work but as title says, I don't have a full licence. I'm pretty screwed as winter is no joke in Canada apparently. Does anyone know if I'll have to wait a year on a G1 or if they will waive the wait period and I can do the test?\n\nI've had my provisional for 8 years and I am 26 btw.\n\nI'll buy anyone who answers this question a beer when I move out.",
  'https://www.reddit

In [66]:
attemptWithfiftyLimit = pd.DataFrame(submissionsData, columns=submissionsCol)

In [68]:
attemptWithfiftyLimit.head()

,id,title,subreddit,author,score,upvote_ratio,num_comments,selftext,url,created_utc
0,t3_ebqeu3,"Ford government cancels Hamilton LRT, starts w...",ontario,santaclo,3,1.00,0,,https://torontosun.com/news/provincial/ford-go...,1.576555e+09
1,t3_ebpsja,Car Insurance question. Looking for any case l...,ontario,Area51Resident,1,0.67,9,Looking for any advise/insight on car insuranc...,https://www.reddit.com/r/ontario/comments/ebps...,1.576552e+09
2,t3_ebpfca,Doug Ford won’t answer your calls anymore. He ...,ontario,anhourisenough,3,0.63,5,,https://www.thestar.com/politics/political-opi...,1.576550e+09
3,t3_ebpa4m,SCRAWLER IN OSHAWA: GM closing may impact enti...,ontario,PolicyWonk7,1,0.67,3,,https://torontosun.com/news/local-news/scrawle...,1.576549e+09
4,t3_ebosc4,How long can a landlord hold a rent cheque for?,ontario,flakula,3,1.00,2,"I know cheques are good for up to 6 months, bu...",https://www.reddit.com/r/ontario/comments/ebos...,1.576547e+09


In [71]:
attemptwithNoLimit = pd.DataFrame(submissionsData, columns=submissionsCol) 

In [72]:
attemptwithNoLimit.head()

,id,title,subreddit,author,score,upvote_ratio,num_comments,selftext,url,created_utc
0,t3_ebqeu3,"Ford government cancels Hamilton LRT, starts w...",ontario,santaclo,3,1.00,0,,https://torontosun.com/news/provincial/ford-go...,1.576555e+09
1,t3_ebpsja,Car Insurance question. Looking for any case l...,ontario,Area51Resident,1,0.67,9,Looking for any advise/insight on car insuranc...,https://www.reddit.com/r/ontario/comments/ebps...,1.576552e+09
2,t3_ebpfca,Doug Ford won’t answer your calls anymore. He ...,ontario,anhourisenough,2,0.56,5,,https://www.thestar.com/politics/political-opi...,1.576550e+09
3,t3_ebpa4m,SCRAWLER IN OSHAWA: GM closing may impact enti...,ontario,PolicyWonk7,1,0.67,3,,https://torontosun.com/news/local-news/scrawle...,1.576549e+09
4,t3_ebosc4,How long can a landlord hold a rent cheque for?,ontario,flakula,3,1.00,2,"I know cheques are good for up to 6 months, bu...",https://www.reddit.com/r/ontario/comments/ebos...,1.576547e+09


In [77]:
print(attemptwithNoLimit == attemptWithfiftyLimit)
print(attemptwithNoLimit.iloc[2])
attemptWithfiftyLimit.iloc[2]

      id  title  subreddit  author  score  upvote_ratio  num_comments  \
0   True   True       True    True   True          True          True   
1   True   True       True    True   True          True          True   
2   True   True       True    True  False         False          True   
3   True   True       True    True   True          True          True   
4   True   True       True    True   True          True          True   
5   True   True       True    True   True          True          True   
6   True   True       True    True   True          True          True   
7   True   True       True    True   True         False          True   
8   True   True       True    True   True         False          True   
9   True   True       True    True  False         False          True   
10  True   True       True    True   True         False          True   
11  True   True       True    True  False         False          True   
12  True   True       True    True  False          

id                                                      t3_ebpfca
title           Doug Ford won’t answer your calls anymore. He ...
subreddit                                                 ontario
author                                             anhourisenough
score                                                           3
upvote_ratio                                                 0.63
num_comments                                                    5
selftext                                                         
url             https://www.thestar.com/politics/political-opi...
created_utc                                           1.57655e+09
Name: 2, dtype: object

In [63]:
totalSubmissionsData = pd.DataFrame(submissionsData, columns=submissionsCol)

print(totalSubmissionsData.head())
print(len(totalSubmissionsData))

SyntaxError: invalid syntax (<ipython-input-63-5fb8ee1635e7>, line 32)

In [47]:
totalSubmissionsData = pd.concat([totalSubmissionsData, submissionsData], axis=1)



In [61]:
submissionsData.head()

,id,title,subreddit,author,score,upvote_ratio,num_comments,selftext,url,created_utc,0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t3_ebqeu3
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Ford government cancels Hamilton LRT, starts w..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ontario
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,santaclo
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [24]:
commentsCol = ["id", "subreddit", "submission_id", "author",\
              "body", "score", "created_utc", "conversation_id"]
conversationCol = ["id", "top_parent_id", "direct_parent_id",\
                  "child_comment_id"]


In [61]:
from collections import deque

def getReplyData(cf, top_level_id, convo_id):
    """
    comment: commentForest Object
    top_level_id: id of the top level comment
    return: tuple of list of lists: (lista, listb), where lista
            is a list of commentData, and listb is a list of convoData
    """
    cf.replace_more(limit=None)
    commentData = []
    convoData = []
    
    for com in list(cf):
        commentData.append([com.name, com.subreddit.display_name, com.submission.id,\
                            com.author.name, com.body, com.score,\
                            convo_id, com.created_utc])
        
        convoData.append([convo_id, top_level_id, com.parent_id, com.id])
        
        # according to documentation, need to call refresh before accessing reply info
        com.refresh()
        
        replyCommentData, replyConvoData = getReplyData(com.replies, top_level_id, convo_id)
        
        commentData += replyCommentData
        convoData += replyConvoData
        
    return (commentData, convoData)
    
def getCommentsFromSubmission(subm_id):
    """
    subm_id: submission id
    Return: list of comment objects from given submisison
    """
    commentData = []
    convoData = []
    # we do this indexing subm_id[2:] because subm_id is
    # the full id of a submission, of the form t2_xxxxx"
    # this reddit.submission method doesnt require the prefix t2_
    subm = reddit.submission(id=subm_id[3:])
    print(subm.title)
    
    # get rid of 'MoreComments' instances
    subm.comments.replace_more(limit=None)
    # top level comments added to deque datastructure
    
    #comments = deque(subm.comments[:])
    comments = subm.comments


    convo_id = 0

#     while len(comments) > 0:
    for com in comments:
        #com = comments.pop
        commentData.append([com.name, com.subreddit.display_name, subm_id,\
                com.author.name, com.body, com.score,\
                convo_id, com.created_utc])
        
        convoData.append([convo_id, None, None, com.id])
        
#         if top_level_id == None:
#             conversation_data = \[convo_id, ]

#         top_level_id = com.name if top_level_id != None
        top_level_id = com.name
        
        com.refresh()
        replyCommentData, replyConvoData = getReplyData(com.replies, top_level_id, convo_id)
        
        commentData += replyCommentData
        convoData += replyConvoData
      
        convo_id += 1
    
    return (commentData, convoData)

In [62]:
pprint.pprint(getCommentsFromSubmission('t3_e9hsir'))

Ontario staring down a $7-billion bill for education talks: Lecce
([['t1_faj3e53',
   'ontario',
   't3_e9hsir',
   'TwitchyJC',
   'The number goes up every time. \n'
   '\n'
   '&#x200B;\n'
   '\n'
   'Also I love the bit at the end about reaching an agreement with EWAO. '
   "Lecce mentions it's an example of him being reasonable at the bargaining "
   'table. What a laughable statement.',
   8,
   0,
   1576123352.0],
  ['t1_faj09kn',
   'ontario',
   't3_e9hsir',
   'CHoDub',
   'I hope you know this is an absolutely false claim... even based on '
   'exaggeration.',
   13,
   1,
   1576120965.0],
  ['t1_faj0n7o',
   'ontario',
   't3_e9hsir',
   'northernontario1',
   'lecce is turning this into a farce',
   9,
   2,
   1576121247.0],
  ['t1_fajxd5q',
   'ontario',
   'e9hsir',
   'ILikeStyx',
   '> lecce is ~~turning this into~~ a farce',
   5,
   2,
   1576156700.0],
  ['t1_fajzlih',
   'ontario',
   'e9hsir',
   'CHoDub',
   "To be honest, I think he's doing a great job.  He w

In [64]:
# create empty df
commentsDataDf = pd.DataFrame([], columns = commentsCol)
conversationDataDf = pd.DataFrame([], columns = conversationCol)

for i, row in submissionsDf.iterrows():
    subm_id = row.id
    comments, convo = getCommentsFromSubmission(subm_id)
    
    commentsDataDf = commentsDataDf.append(comments)
    conversationDataDf = conversationDataDf.append(convo)
    break

Ontario staring down a $7-billion bill for education talks: Lecce


In [65]:
print(len(commentsDataDf))
print(len(conversationDataDf))

16
16
